Fase 4: Extracción de Tablas de los Premios Oscar con Beautiful Soup
En esta fase, trabajarán con la biblioteca Beautiful Soup para extraer información relevante de las tablas de los premios Oscar desde 2000. La tabla la encontrarás en el este link.https://es.wikipedia.org/wiki/Premios_%C3%93scar La información que deberás sacar es:

Fecha de la ceremonia.

Mejor película.

Mejor director.

Mejor actor.

Mejor actriz.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

Copy
  [(1990, 'Driving Miss Daisy', 'O. Stone', "D. Day-Lewis",' J. Tandy' ),  ...]

In [1]:
#importamos las librerías necesarias
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [2]:
#definimos la url, hacemos la llamada y comprobamos el status.
url_oscars='https://es.wikipedia.org/wiki/Premios_%C3%93scar'
res_oscars=requests.get(url_oscars)
res_oscars.status_code

200

In [3]:
#creamos la sopa
sopa_oscars=BeautifulSoup(res_oscars.content, 'html.parser')
print(sopa_oscars)

<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-enabled vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-disabled skin-theme-clientpref-day vector-toc-available" dir="ltr" lang="es">
<head>
<meta charset="utf-8"/>
<title>Premios Óscar - Wikipedia, la enciclopedia libre</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-d

In [4]:
#buscamos las tablas de la sopa
tablas_wiki=sopa_oscars.find_all('table')
len(tablas_wiki) #vemos que hay 4 tablas
tablas_wiki[2] #probando, hemos visto que la tabla que necesitamos es la tercera
tabla_oscars=tablas_wiki[2] #guardamos la tabla en una variable, para saber a cuál referirnos

In [5]:
#buscamos y guardamos los encabezados en una lista
lista_heads=tabla_oscars.find_all('th')
print(lista_heads)
lista_heads_texto=[columna.text for columna in lista_heads]
print(f'Los encabezados son: {lista_heads_texto}')

[<th style="width:1px;">N.º
</th>, <th style="width:140px;">Ceremonia
</th>, <th style="width:250px;">Mejor película
</th>, <th style="width:225px;">Mejor director
</th>, <th style="width:225px;">Mejor actor
</th>, <th style="width:225px;">Mejor actriz
</th>, <th style="width:220px;">Mejor actor reparto
</th>, <th style="width:220px;">Mejor actriz reparto
</th>, <th style="width:170px;">Presentador
</th>, <th>Audiencia
</th>]
Los encabezados son: ['N.º\n', 'Ceremonia\n', 'Mejor película\n', 'Mejor director\n', 'Mejor actor\n', 'Mejor actriz\n', 'Mejor actor reparto\n', 'Mejor actriz reparto\n', 'Presentador\n', 'Audiencia\n']


In [6]:
#quitamos los \n (hay 10 columnas)
lista_heads_texto=[x.rstrip() for x in lista_heads_texto]
lista_heads_texto

['N.º',
 'Ceremonia',
 'Mejor película',
 'Mejor director',
 'Mejor actor',
 'Mejor actriz',
 'Mejor actor reparto',
 'Mejor actriz reparto',
 'Presentador',
 'Audiencia']

In [7]:
#nos quedamos sólo con las columnas que nos interesan
heads_oscars=lista_heads_texto[1:6]
heads_oscars

['Ceremonia',
 'Mejor película',
 'Mejor director',
 'Mejor actor',
 'Mejor actriz']

In [8]:
#guardamos las filas en una lista
filas_oscars=tabla_oscars.find_all('tr')
filas_oscars[1].text #vemos las primeras para comprobar que están ok

'\n1°\n\n16 de mayode 1929\n\n Alas (Wings) (1927)\n\nDrama:  F. BorzageCom.:  L. Milestone\n\n E. Jannings(The Last Command)\n\n J. Gaynor(7th Heaven)\n\n—\n\n—\n\nDouglas FairbanksWilliam C. deMille\n\n\n'

In [9]:
#quitamos los \n y nos quedamos sólo con las columnas que nos interesan
filas_oscars_2 = []
for i in filas_oscars[1:]:
    limpieza = i.text.split("\n")
    filas_oscars_2.append(limpieza[3:12:2])

print(filas_oscars_2[0])

['16 de mayode 1929', ' Alas (Wings) (1927)', 'Drama:  F. BorzageCom.:  L. Milestone', ' E. Jannings(The Last Command)', ' J. Gaynor(7th Heaven)']


In [10]:
#del resultado anterior, queremos únicamente el año y el principio de las demás celdas, no queremos todo lo que viene entre paréntesis
for indice_lista, lista in enumerate(filas_oscars_2): #recorremos cada fila
    for indice_elemento, elemento in enumerate(lista): #recorremos cada celda de la fila
        if indice_elemento == 0: #si es el primero (fecha ceremonia)
            anio=re.search(r'\d{4}', elemento) #buscamos y cogemos sólo el año (dígito 4 cifras)
            filas_oscars_2[indice_lista][indice_elemento]=anio.group()
        else:
            #separamos cuando haya un paréntesis y nos quedamos con el principio
            separacion=elemento.split('(')
            filas_oscars_2[indice_lista][indice_elemento]=separacion[0]
            #eliminamos 'Drama: ' y 'Com'
            filas_oscars_2[indice_lista][indice_elemento]= filas_oscars_2[indice_lista][indice_elemento].replace('Drama: ','').replace('Com','')

print(filas_oscars_2[0])

['1929', ' Alas ', ' F. Borzage.:  L. Milestone', ' E. Jannings', ' J. Gaynor']


In [11]:
#comprobamos longitud
longitud=len(filas_oscars_2)
longitud

96

In [12]:
#pasamos cada elemento de la lista filas_oscars_2 a una tupla para que nos quede una lista de tuplas, como pide el enunciado
lista_tuplas_oscars=[]
for i in filas_oscars_2:
    tuplas_oscars=tuple(i)
    lista_tuplas_oscars.append(tuplas_oscars)

print(lista_tuplas_oscars)

[('1929', ' Alas ', ' F. Borzage.:  L. Milestone', ' E. Jannings', ' J. Gaynor'), ('1930', ' La melodía de Broadway ', ' F. Lloyd', ' W. Baxter', ' M. Pickford'), ('1930', ' Sin novedad en el frente ', ' L. MilestoneSin novedad en el frente ', ' G. Arliss', ' N. Shearer'), ('1931', ' Cimarron ', ' N. Taurog', ' L. Barrymore', ' M. Dressler'), ('1932', ' Grand Hotel', ' F. Borzage', ' F. March W. Beery', ' H. Hayes'), ('1934', ' Cavalcade', ' F. Lloyd', ' C. Laughton', ' K. Hepburn'), ('1935', ' It Happened One Night', ' F. Capra', ' C. Gable', ' C. Colbert'), ('1936', ' Mutiny on the Bounty', ' J. Ford', ' V. McLaglen', ' B. Davis'), ('1937', ' The Great Ziegfeld', ' F. Capra', ' P. Muni', ' L. Rainer'), ('1938', ' life of emile !The Life of Emile Zola', ' L. McCarey', ' S. Tracy', ' L. Rainer'), ('1939', " You Can't Take It With You", ' F. Capra', ' S. Tracy', ' B. Davis'), ('1940', ' Gone with the Wind', ' V. Fleming', ' R. Donat', ' V. Leigh'), ('1941', ' Rebecca', ' J. Ford', ' J. 

In [45]:
#selector de la tabla que queremos:
#mw-content-text > div.mw-content-ltr.mw-parser-output > table.wikitable

In [14]:
def datos_oscars(url):
    #hacemos la llamada y comprobamos estado de la misma
    res_oscars=requests.get(url)
    print(f'La respuesta a la petición es: {res_oscars.status_code}')

    #creamos la sopa
    sopa_oscars=BeautifulSoup(res_oscars.content, 'html.parser')

    #selector de la tabla que queremos
    tabla_oscars=sopa_oscars.find('table', {'class':'wikitable'})

    #buscamos y guardamos los encabezados
    lista_heads=tabla_oscars.find_all('th') #buscamos los encabezados
    lista_heads_texto=[columna.text for columna in lista_heads] #extraemos sólo el texto
    lista_heads_texto=[x.rstrip() for x in lista_heads_texto] #quitamos los \n
    heads_oscars=lista_heads_texto[1:6] #nos quedamos sólo con los encabezados que nos interesan
    
    #buscamos y guardamos las filas
    filas_oscars=tabla_oscars.find_all('tr') #buscamos las listas
    #quitamos los \n y nos quedamos sólo con las columnas que nos interesan
    filas_oscars_2 = []
    for i in filas_oscars[72:]: #ponemos desde la fila 72, que corresponde al año 2000
        limpieza = i.text.split("\n")
        filas_oscars_2.append(limpieza[3:12:2])
    #del resultado anterior, queremos únicamente el año y el principio de las demás celdas, no queremos todo lo que viene entre paréntesis
    for indice_lista, lista in enumerate(filas_oscars_2): #recorremos cada fila
        for indice_elemento, elemento in enumerate(lista): #recorremos cada celda de la fila
            if indice_elemento == 0: #si es el primero (fecha ceremonia)
                anio=re.search(r'\d{4}', elemento) #buscamos y cogemos sólo el año (dígito 4 cifras)
                filas_oscars_2[indice_lista][indice_elemento]=anio.group() #.group() retorna la cadena de caracteres que coincide con la re
            else:
                #separamos cuando haya un paréntesis y nos quedamos con el principio
                separacion=elemento.split('(')
                filas_oscars_2[indice_lista][indice_elemento]=separacion[0]
                #eliminamos 'Drama: ' y 'Com'
                filas_oscars_2[indice_lista][indice_elemento]= filas_oscars_2[indice_lista][indice_elemento].replace('Drama: ','').replace('Com','')

    #pasamos cada elemento de la lista filas_oscars_2 a una tupla para que nos quede una lista de tuplas, como pide el enunciado
    lista_tuplas_oscars=[]
    for i in filas_oscars_2:
        tuplas_oscars=tuple(i)
        lista_tuplas_oscars.append(tuplas_oscars)

    return heads_oscars, lista_tuplas_oscars

In [15]:
url_oscars='https://es.wikipedia.org/wiki/Premios_%C3%93scar'

In [16]:
#llamamos a la función
heads_oscars, lista_tuplas_oscars = datos_oscars(url_oscars)

La respuesta a la petición es: 200


In [17]:
#creamos DataFrame del resultado
df_oscars=pd.DataFrame(lista_tuplas_oscars)
df_oscars.columns=heads_oscars
df_oscars

,Ceremonia,Mejor película,Mejor director,Mejor actor,Mejor actriz
0,2000,American Beauty,S. Mendes,K. Spacey,H. Swank
1,2001,Gladiator,S. Soderbergh,R. Crowe,J. Roberts
2,2002,A Beautiful Mind,R. Howard,D. Washington,H. Berry
3,2003,Chicago,R. Polanski,A. Brody,N. Kidman
4,2004,The Lord of the Rings: The Return of the King,P. Jackson,S. Penn,C. Theron
5,2005,Million Dollar Baby,C. Eastwood,J. Foxx,H. Swank
6,2006,Crash,A. Lee,P. S. Hoffman,R. Witherspoon
7,2007,The Departed,M. Scorsese,F. Whitaker,H. Mirren
8,2008,No Country for Old Men,J. Coen E. Coen,D. Day-Lewis,M. Cotillard
9,2009,Slumdog Millionaire,D. Boyle,S. Penn,K. Winslet


In [18]:
#consultamos los últimos
df_oscars.tail()

,Ceremonia,Mejor película,Mejor director,Mejor actor,Mejor actriz
20,2020,Parasite,Bong J.,J. Phoenix,R. Zellweger
21,2021,Nomadland,C. Zhao,A. Hopkins,F. McDormand
22,2022,CODA,J. Campion,W. Smith,J. Chastain
23,2023,Everything Everywhere All at Once,D. Kwan D. Scheinert,B. Fraser,M. Yeoh
24,2024,Oppenheimer,C. Nolan,C. Murphy,E. Stone


In [19]:
#exportamos tabla a csv para su futura incorporación a la BDD
df_oscars.to_csv("Fase4.csv")